# 数据表检查和数据清洗

##  第二章 数据检查

### 导入Excel表

In [ ]:
# 导入Excel表
import pandas as pd
df = pd.read_excel(r'维护单201902测试.xlsx',sheet_name=0 )
df.head(3)

### 指定条件拆分一个Excel工作表成多个Excel文件
- 先把原始文件用pandas读取，按分公司分割成多个Excel文件  
- 再利用循环读取一个保存分公司片区的字典表，对每个分公司的明细进行进一步处理

In [161]:
child_wb = df[df['所属区域'] == '宜州市']
child_wb.loc[:,['片区']] = child_wb.loc[:,['片区']] .fillna(value='')
pqarea = child_wb['片区'].unique()
pqarea[0]

'河池'

In [ ]:
child_wb = df.loc[df['所属区域'] == '宜州市']
child_wb.loc[:,['片区']] = child_wb.loc[:,['片区']] .fillna(value='')
child_wb.head(3)
#pqarea

In [162]:
# 拆分后的文件放置位置，路径的最后一个字符必须是空格？
result_path =  r'C:\Users\12112\Documents\明细拆分\ '
# 获取拆分条件：所属区域(分公司),去重
area_list = df['所属区域'].unique()
# 按条件拆分后分别保存新的Excel文件
for area in area_list:
# 按所属区域选择多行
    child_wb = df[df['所属区域'] == area]
    child_wb.loc[:,['片区']] = child_wb.loc[:,['片区']] .fillna(value='')
    pqarea = child_wb['片区'].unique()
    child_wb.to_excel(result_path + pqarea[0] +'片区_'+ area + '.xlsx',sheet_name='工单明细',index = False)
print('拆分完成')

拆分完成


### Pandas连接MySQL数据库

In [ ]:
import pymysql
import pandas as pd
import numpy as np
# 打开数据库连接
dbcon = pymysql.connect(host='10.0.28.20',user='root',passwd='root96335',charset='UTF8',db='gdata')
sql = 'select * from whdtsb_201903'
df2 = pd.read_sql(sql, dbcon)
# 关闭数据库连接
dbcon.close()

# 用map+匿名格式化函数，把小数格式化为带 % 号的字符串，可以导出为字符串，但是不利于进一步分析数据，丢失了部分小数位还要再做一次转换回浮点数
df2['满意率'] = (df2['满意']/df2['评价数']).map(lambda x:format(x,'.2%'))
df2['城网工单处理及时率'] = (df2['城网未超时工单数']/df2['工单总数']).map(lambda x:format(x,'.2%'))
df2

# 只是在Dataframe显示时格式化为百分比，但是导出后还是浮点数格式
df2['满意率'] = (df2['满意']/df2['评价数'])
df2 = df2.style.format({ '满意率': '{:,.2%}'.format})
df2

`DataFrame`保存为Excel文件，默认的文件保存位置和ipynb文件在同一个路径下

In [61]:
df2.to_excel('whdtsb_201903.xlsx',sheet_name='201903透视表汇总')

#### 从数据库中导出明细并按分公司分割为多个Excel文件

In [165]:
import pymysql
import pandas as pd
import numpy as np
# 打开数据库连接
dbcon = pymysql.connect(host='10.0.28.20',user='root',passwd='root96335',charset='UTF8',db='gdata')
sql = 'select * from 维护工单抽取'
df_whgd = pd.read_sql(sql, dbcon)
# 拆分后的文件放置位置，路径的最后一个字符必须是空格？
result_path =  r'C:\Users\12112\Documents\mysql维护工单抽取明细拆分\ '
# 获取拆分条件：所属区域(分公司),去重
area_list = df_whgd['所属区域'].unique()
# 按条件拆分后分别保存新的Excel文件
for area in area_list:
# 按所属区域选择多行
    child_wb = df_whgd[df_whgd['所属区域'] == area]
    child_wb.loc[:,['片区']] = child_wb.loc[:,['片区']] .fillna(value='')
    pqarea = child_wb['片区'].unique()
    child_wb.to_excel(result_path + pqarea[0] +'片区_'+ area + '.xlsx',sheet_name='工单明细',index = False)
print('拆分完成')
# 关闭数据库连接
dbcon.close()

C:\ProgramData\Miniconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\ProgramData\Miniconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


拆分完成


### 查看数据表信息

In [ ]:
#数据表信息
df.info()

### 查看数据表各列格式

In [ ]:
#查看数据表各列格式
df.dtypes

### 查看单列格式

In [ ]:
#查看单列格式
df['城网是否超时'].dtypes

### 对整个表检查数据空值

In [ ]:
#对整个表检查数据空值
df.isnull()

### 检查特定列空值

In [ ]:
#检查特定列空值
df['工单原因1'].isnull()

### 删除空行

In [ ]:
df.dropna(how='all')

### 查看某列中的唯一值

In [74]:
#查看 某列中的唯一值，相当于去重之后的效果
df['客户姓名'].unique()

array(['谭本荣', '李海燕', '周国增', ..., '黄益民', '陆锦秀', '黎水兰'], dtype=object)

### 查看数据表的值，以数组形式返回

In [ ]:
# 查看数据表的值，以数组形式返回
df.values

### 查看列名称

In [33]:
#查看列名称
df.columns

Index(['工单编号', '工单来源', '片区', '所属区域', '来电号码', '受理编号', '客户姓名', '网络类型', '客户地址',
       '第一次到分公司调度环节', '客户类型', 'VIP客户等级', '工单类型', '服务类型', '分公司处理部门', '分公司回单工号',
       '分公司处理人员', '最后一次到呼叫中心回访环节', '分公司回单内容', '回访时间', '回访结果', '工单满意', '不满意原因',
       '不满意对象', '满意度定性', '是否及时预约', '回访处理意见', '投诉有效性', '投诉性质', '投诉定性', '投诉原因',
       '投诉对象', '定性人', '是否申请延时', '延时时长', '问题是否解决', '第一次到工单处理环节', '撤销时间', '是否撤销',
       '环节历时', '是否12小时办结', '是否24小时办结', '是否超48小时办结', '是否48小时办结', '最后环节',
       '需求描述1', '需求描述2', '需求描述3', '需求描述4', 'BOSS区域', '网格编码', '网格名称', '维护人员',
       '工单原因1', '工单原因2', '工单原因3', '机顶盒号', '是否6小时办结', '环节历时2', '是否18时前受理',
       '城网是否超时', '农网是否超时'],
      dtype='object')

### 查看前3行数据

In [ ]:
# 查看前3行数据
df.head(5)

### 查看最后3行

In [ ]:
#查看最后3行
df.tail(5)

## 第三章 数据清洗
### 填充数据表中空值

In [ ]:
#填充数据表中空值
df.fillna(value='')

### 清除city字段中的字符空格

In [ ]:
#清除city字段中的字符空格
df['所属区域']=df['所属区域'].map(str.strip)
df['所属区域']

### 更改列数据格式

In [ ]:
df['第一次到分公司调度环节'] = df['第一次到分公司调度环节'].fillna(value=0) # 将空值用0来填充
# 更改列数据格式
df['第一次到分公司调度环节'].astype('datetime64')  #  这样的操作并没有改变原始的数据框，而只是返回的一个拷贝
df.dtypes   #这里会查看到类型没有发生变化

In [ ]:
# 更改列数据格式
# 想要真正的改变数据框，通常需要通过赋值来进行，比如
df['第一次到分公司调度环节'] = df['第一次到分公司调度环节'].astype('datetime64')
df.dtypes

还有两种更可用的方法：
- 自定义函数
- 使用`pd.to_numeric()`，`pd.to_datetime()`函数

### 更改列名称

In [ ]:
# 更改列名称，要真正地改变列名称，需要通过赋值来进行：
df = df.rename(columns={'所属区域':'分公司'})
df.head(3)

### 删除重复值
默认情况下drop_duplicates()将删除后出现的重复值(与Excel逻辑一致)。增加keep='last'参数后将删除最先出现的重复值，保留最后的值。

In [ ]:
df['来电号码'].drop_duplicates()

### 数值修改及替换

In [ ]:
df['客户姓名'].replace('谭本荣','谭本荣2')

## 第四章 数据预处理
### 数据表合并

In [102]:
df2=pd.read_excel(r'ExceltoPython.xlsx',sheet_name='Sheet1')
df2

,id,date,city,category-size,age,price
0,1001,2013-01-02,bejing,100-A,23,1200
1,1002,2013-01-03,sh,100-B,44,3299
2,1003,2013-01-04,guangzhou,110-A,54,2133
3,1004,2013-01-05,shenzhen,110-C,32,5433
4,1005,2013-01-06,shanghai,201-A,34,3299
5,1009,2013-01-07,beijing,130-F,32,4432


In [101]:
df1=pd.read_excel(r'ExceltoPython.xlsx',sheet_name='Sheet2')
df1

,id,gender,pay,m-point
0,1001,male,Y,10
1,1002,female,N,12
2,1003,male,Y,20
3,1004,female,Y,40
4,1005,male,N,40
5,1006,female,Y,40
6,1007,male,N,30
7,1008,female,Y,20


#### 内连接

In [103]:
df_inner = pd.merge(df2,df1,how='inner',on=['id'])
df_inner

,id,date,city,category-size,age,price,gender,pay,m-point
0,1001,2013-01-02,bejing,100-A,23,1200,male,Y,10
1,1002,2013-01-03,sh,100-B,44,3299,female,N,12
2,1003,2013-01-04,guangzhou,110-A,54,2133,male,Y,20
3,1004,2013-01-05,shenzhen,110-C,32,5433,female,Y,40
4,1005,2013-01-06,shanghai,201-A,34,3299,male,N,40


#### 左外连接

In [104]:
df_left = pd.merge(df2,df1,how='left',on=['id'])
df_left

,id,date,city,category-size,age,price,gender,pay,m-point
0,1001,2013-01-02,bejing,100-A,23,1200,male,Y,10.0
1,1002,2013-01-03,sh,100-B,44,3299,female,N,12.0
2,1003,2013-01-04,guangzhou,110-A,54,2133,male,Y,20.0
3,1004,2013-01-05,shenzhen,110-C,32,5433,female,Y,40.0
4,1005,2013-01-06,shanghai,201-A,34,3299,male,N,40.0
5,1009,2013-01-07,beijing,130-F,32,4432,NaN,NaN,NaN


#### 右外连接

In [105]:
df_right = pd.merge(df2,df1,how='right',on=['id'])
df_right

,id,date,city,category-size,age,price,gender,pay,m-point
0,1001,2013-01-02,bejing,100-A,23.0,1200.0,male,Y,10
1,1002,2013-01-03,sh,100-B,44.0,3299.0,female,N,12
2,1003,2013-01-04,guangzhou,110-A,54.0,2133.0,male,Y,20
3,1004,2013-01-05,shenzhen,110-C,32.0,5433.0,female,Y,40
4,1005,2013-01-06,shanghai,201-A,34.0,3299.0,male,N,40
5,1006,NaT,NaN,NaN,NaN,NaN,female,Y,40
6,1007,NaT,NaN,NaN,NaN,NaN,male,N,30
7,1008,NaT,NaN,NaN,NaN,NaN,female,Y,20


#### 全连接
indicator的参数，如果置True的时候，输出结果会增加一列 ` _merge`，表示在哪些表中存在

In [111]:
df_outer = pd.merge(df2,df1,how='outer',on=['id'],indicator=True)
df_outer

,id,date,city,category-size,age,price,gender,pay,m-point,_merge
0,1001,2013-01-02,bejing,100-A,23.0,1200.0,male,Y,10.0,both
1,1002,2013-01-03,sh,100-B,44.0,3299.0,female,N,12.0,both
2,1003,2013-01-04,guangzhou,110-A,54.0,2133.0,male,Y,20.0,both
3,1004,2013-01-05,shenzhen,110-C,32.0,5433.0,female,Y,40.0,both
4,1005,2013-01-06,shanghai,201-A,34.0,3299.0,male,N,40.0,both
5,1009,2013-01-07,beijing,130-F,32.0,4432.0,NaN,NaN,NaN,left_only
6,1006,NaT,NaN,NaN,NaN,NaN,female,Y,40.0,right_only
7,1007,NaT,NaN,NaN,NaN,NaN,male,N,30.0,right_only
8,1008,NaT,NaN,NaN,NaN,NaN,female,Y,20.0,right_only


### 设置索引列

In [113]:
df_inner.set_index(['id','age'])

,,date,city,category-size,price,gender,pay,m-point
id,age,,,,,,,
1001,23,2013-01-02,bejing,100-A,1200,male,Y,10
1002,44,2013-01-03,sh,100-B,3299,female,N,12
1003,54,2013-01-04,guangzhou,110-A,2133,male,Y,20
1004,32,2013-01-05,shenzhen,110-C,5433,female,Y,40
1005,34,2013-01-06,shanghai,201-A,3299,male,N,40
